In [ ]:
#imports
import pandas as pd
import numpy as np
import gurobipy as gp
#from mip import *

In [ ]:
#teacher evaluation --> 1 - Improvement required (not the sharpest pencil), 10 - Excellent, per subject
subject_and_teacher, teacher_evaluation = gp.multidict({
    'Bible_Roni': 5,
    'Bible_Rachel': 5,
    'Literature_Sharon': 6,
    'Literature_Liora': 6,
    'Math_Yuri': 3,
    'Math_Sarit': 3,
    'Math_Victor': 3,
    'English_Miriam': 7,
    'English_Haim': 7,
    'Physics_Tzvika': 2,
    'Language_Hanita': 4,
    'Language_Smadar': 4,
    'Geography_Ilana': 10,
    'History_Ruth': 6,
    'History_Etti': 6,
    'Chemistry_Shelly': 7,
    'CitizenshipLesson_Tami': 8,
    'Economics_Menachem': 5,
    'Arabic_Hanita': 9
})

In [ ]:
#subject + avg grade
student_avg = gp.multidict({
    'Bible_Roni': 70,
    'Bible_Rachel': 70,
    'Literature_Sharon': 72,
    'Literature_Liora': 72,
    'Math_Yuri': 53,
    'Math_Sarit': 53,
    'Math_Victor': 53,
    'English_Miriam': 68,
    'English_Haim': 68,
    'Physics_Tzvika': 41,
    'Language_Hanita': 80,
    'Language_Smadar': 80,
    'Geography_Ilana': 94,
    'History_Ruth': 78,
    'History_Etti': 78,
    'Chemistry_Shelly': 75,
    'CitizenshipLesson_Tami': 86,
    'Economics_Menachem': 56,
    'Arabic_Hanita': 66
})[1]

In [ ]:
teacher_free_time = {
    'Sunday': [['Bible_Rachel', 10, 12], ['Literature_Liora', 14, 16]],
    'Monday': [['Literature_Sharon', 13, 15], ['Math_Yuri', 12, 14], ['English_Haim', 17, 19]],
    'Tuesday': [['Physics_Tzvika', 16, 18], ['Language_Hanita', 14, 16],
                ['Economics_Menachem', 14, 16], ['English_Miriam', 10, 12]],
    'Wednesday': [['Language_Smadar', 9, 11], ['Geography_Ilana', 13, 15],
                  ['History_Ruth', 12, 14], ['CitizenshipLesson_Tami', 15, 17]],
    'Thursday': [['Math_Sarit', 9, 11], ['Math_Victor', 17, 19], ['Arabic_Hanita', 18, 20]],
    'Friday': [['Bible_Roni', 8, 10], ['History_Etti', 10, 12], ['Chemistry_Shelly', 11, 13]]
}

In [ ]:
student_free_time = {
    'Sunday': [14,20],
    'Monday': [12,15],
    'Tuesday': [16,19],
    'Wednesday': [12,17],
    'Thursday': [17,20],
    'Friday': [8,13]
}

In [ ]:
#find the subjects which their time collides while checking the days and houres
#teacher_free_time is when the classes occurs
def get_collisions(teacher_free_time):
    collisions = []
    for subjects_per_day in teacher_free_time.values():
        # subjects_per_day is a list of all the subjects in each day
        if len(subjects_per_day) == 1:
            continue  # there are no possible collisions because there is only one subject a day.

        # in case there is more than one subjects per day, check if the class times collides by comparing start and end times.
        for i, course in enumerate(subjects_per_day):
            if i == len(subjects_per_day) - 1:
                continue  # all collisions were already specified before
            for other_course in subjects_per_day[i + 1:]:
                start_time_current = course[1]
                end_time_current = course[2]
                start_time_other = other_course[1]
                course_name = course[0]
                other_course_name = other_course[0]
                if (start_time_other < end_time_current) or (start_time_other == start_time_current):
                    collisions.append((course_name, other_course_name))

    return collisions

collisions = get_collisions(teacher_free_time)

In [ ]:
print(collisions)

In [ ]:
#find the subjects which their time does not match with the student schedule
def get_inopportune_classes(teacher_free_time, student_free_time):
    #a list to hold all the classes that not fit with the student schedule
    inopportune_classes = []
    #loop over the two dictinaries at once
    for (student_day, s_free_time), (day, day_subjects) in zip(student_free_time.items(), teacher_free_time.items()):
        day_length = student_free_time[student_day]     
        for subject in day_subjects:
            start = subject[1]
            end = subject[2]
            if start < day_length[0] or end > day_length[1]:
                inopportune_classes.append(subject[0])

    return inopportune_classes

inopportune_classes = get_inopportune_classes(teacher_free_time, student_free_time)

In [ ]:
print(inopportune_classes)

In [ ]:
#some subjects are the same with different teachers (like bible, math..)
#we want the student to be able to enroll to a subject only once (and ont once per teacher).
def get_same_classes(teacher_free_time):
    same_subjects = {}
    for day in teacher_free_time.values():
        #for example: subject --> ['Bible_Rachel', 10, 12]
        for subject in day:
            #subject[0] --> Bible_Rachel
            split_sub_and_teacher = subject[0].split("_")
            #split_sub_and_teacher --> Bible,Rachel
            if split_sub_and_teacher[0] in same_subjects:
                same_subjects[split_sub_and_teacher[0]].append(subject)
            else:
                same_subjects[split_sub_and_teacher[0]] = [subject]
    return same_subjects

same_subjects = get_same_classes(teacher_free_time)

In [ ]:
print(same_subjects)

In [ ]:
import time
import datetime
start = time.time()
start = datetime.datetime.now()

# Creating the model
model = gp.Model("Partani")

# Create decision variables for the subjects the student should take as 'partani' houres.
subjects_to_assign = model.addVars(subject_and_teacher, name="Subjects+Teachers", vtype=gp.GRB.BINARY)

In [ ]:
# objective
# The objective is to minimize (lowest grades and min teacher evaluation)

model.ModelSense = gp.GRB.MINIMIZE

evaluation_objective = subjects_to_assign.prod(teacher_evaluation)
evaluation_priority = 0
evaluation_weight = 2

student_avg_objective = subjects_to_assign.prod(student_avg)
student_avg_priority = 0
student_avg_weight = 1

model.setObjectiveN(evaluation_objective, evaluation_priority, weight=evaluation_weight)
model.setObjectiveN(student_avg_objective, student_avg_priority, weight=student_avg_weight)

In [ ]:
def optimize(subjects_to_assign, same_subjects, inopportune_classes, collisions):

    for course1, course2 in collisions:
        model.addConstr(subjects_to_assign.sum(course1) + subjects_to_assign.sum(course2) <= 1)

    for course in inopportune_classes:
        model.addConstr(subjects_to_assign.sum(course) == 0)

    for course in same_subjects.values():
        courses_sum = 0
        for course_time in course:
            courses_sum += subjects_to_assign.sum(course_time)
        model.addConstr(courses_sum <= 1)

    model.optimize()

    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % model.objVal)
    end = time.time()
    end = datetime.datetime.now()
    print(end - start)
    
optimize(subjects_to_assign, same_subjects, inopportune_classes, collisions)